In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import timeit

import delfi.generator
from delfi.distribution import Uniform
from delfi.summarystats import Identity
from delfi.simulator.LotkaVolterra import LotkaVolterra

from delfi.inference import SNPEC as APT

from util import StubbornGenerator, stubborn_defaultrej
from util import load_setup_lv as load_setup
from util import load_gt_lv as load_gt
from util import draw_sample_uniform_prior_52 as rej_sampler

from snl.util.plot import plot_hist_marginals

seed = 1305

# simulation setup
setup_dict = load_setup()


# simulator 
include_initial_state = True # initial population [50, 100]
dt = 0.2         # time step

noise_sd = 50.0  # set to 0.0 for no noise. 
                 # compare to log(noise_sd) = 2.3 in 
                 # Owen et al. 2014, Likelihood free inference for Makov Processes: a comparison


pars_true, _ = load_gt(generator=None)
print('pars_true : ', pars_true)

# x0 was generated by simulation using the true parameters from Papamakarios et al. (2018),
# include_initial_state=True, dt=0.2
raw_data = np.array([50., 100., 59., 104.,  66., 117.,  81., 133.,  97., 131., 109., 135., 127.,
       132., 148., 135., 163., 125., 188., 108., 217.,  95., 239.,  75.,
       236.,  60., 235.,  46., 228.,  34., 217.,  28., 211.,  18., 193.,
        12., 175.,   8., 171.,   9., 162.,   5., 151.,   5., 136.,   6.,
       128.,   6., 111.,   8., 110.,   8.,  96.,   8.,  95.,   9.,  93.,
         9.,  81.,  10.,  80.,  11.,  78.,  17.,  74.,  17.,  72.,  19.,
        66.,  20.,  67.,  25.,  64.,  29.,  54.,  30.,  55.,  45.,  54.,
        45.,  53.,  48.,  48.,  50.,  54.,  65.,  61.,  75.,  64.,  77.,
        65.,  95.,  73., 106.,  85., 107.,  88., 114., 104., 113., 130.,
       114., 159., 110., 173.,  95., 196.,  77., 220.,  72., 219.,  49.,
       212.,  33., 209.,  24., 196.,  18., 188.,  18., 176.,  17., 166.,
        15., 158.,  14., 154.,  10., 138.,  13., 128.,  11., 119.,  11.,
       107.,   9.,  99.,   8.,  94.,  11.,  86.,  11.,  81.,  12.,  78.,
        15.,  67.,  19.,  63.,  16.,  58.,  20.,  58.,  28.,  57.,  36.,
        53.,  39.,  55.,  44.,  53.,  44.,  55.,  44.,  54.,  49.,  54.,
        66.,  54.,  77.,  53.,  93.,  64.,  98.,  76., 100.,  94., 110.,
       116., 112., 136., 102., 149., 105., 173., 103., 200.,  95., 208.,
        69., 224.,  54., 232.,  47., 217.,  40., 206.,  29., 202.,  24.,
       196.,  24., 181.,  23., 167.,  13., 158.,  14., 147.,   9., 135.,
         8., 121.,   9., 111.,   8.,  95.,  10.,  84.,  10.,  73.,  11.,
        68.,  14.,  61.,  16.,  57.,  20.,  50.,  19.,  46.,  25.,  43.,
        33.,  41.,  38.,  42.,  40.,  45.,  50.,  48.,  57.,  48.,  60.,
        47.,  63.,  49.,  68.,  48.,  66.,  42.,  71.,  43.,  84.,  46.,
        81.,  40.,  89.,  45., 121.,  55., 134.,  67., 136.,  83., 135.,
       100., 142., 129., 135., 148., 134., 163., 130., 182., 111., 195.,
       106., 206.,  93., 216.,  67., 224.,  47., 207.,  33., 202.,  24.,
       193.,  20., 182.,  14., 174.,   8., 161.,   8., 152.,   4., 140.,
         6., 131.,   5.])

# add observation noise to raw data
raw_data += np.random.RandomState(seed=seed).randn(*raw_data.shape) * noise_sd
obs = raw_data if include_initial_state else raw_data[2:]

# generator object (prior + simulator + summary statistics)
def init_g(seed, n_steps_used=(raw_data.shape[0] - 2)// 2):
    
    assert (n_steps_used + int(include_initial_state)) * 2 <= raw_data.shape[0]
    
    model = LotkaVolterra(dt=dt, duration=dt * n_steps_used, seed=seed, max_n_steps=50000, noise_sd=noise_sd)
    prior = Uniform(lower= [-5,-5,-5,-5], upper = [2,2,2,2], seed=seed)
    g = StubbornGenerator(model=model, prior=prior, summary=Identity(), seed=seed)
    
    ii_max = n_steps_used + 1 if include_initial_state else n_steps_used
    
    return g, ii_max

g, ii_max = init_g(seed=seed)


# fit APT

In [ ]:
setup_dict['proposal'] = 'gaussian' # Gaussian proposal APT
setup_dict['n_inputs_rnn'] = 2 # bivariate observations (predator and prey) at any time point
setup_dict['n_rnn'] = 100      # number of GRU units
setup_dict['epochs'] = 2000     
setup_dict['val_frac'] = 0.1    

In [ ]:
if setup_dict['train_on_all']:
    epochs=[setup_dict['epochs']//(r+1) for r in range(setup_dict['n_rounds'])]
else:
    epochs=setup_dict['epochs']
    
res = APT(g,
          obs=obs[:ii_max * 2],
          n_hiddens=setup_dict['n_hiddens'],
          n_rnn=setup_dict['n_rnn'],
          n_inputs_rnn=setup_dict['n_inputs_rnn'],
          seed=seed,
          pilot_samples=setup_dict['pilot_samples'],
          svi=setup_dict['svi'],
          verbose=setup_dict['verbose'],
          prior_norm=setup_dict['prior_norm'])

print('conditional density estimator', res.network)

# train
t = timeit.time.time()
print('fitting model with SNPC-C')
logs, tds, posteriors = res.run(
                    n_train=setup_dict['n_train'],
                    proposal=setup_dict['proposal'],
                    n_rounds=setup_dict['n_rounds'],
                    train_on_all=setup_dict['train_on_all'],
                    minibatch=setup_dict['minibatch'],
                    moo=setup_dict['moo'],
                    val_frac=setup_dict['val_frac'],
                    epochs=epochs, 
                    verbose=True,
                    silent_fail=False)
print('fitting time : ', timeit.time.time() - t)

# inspect results

In [ ]:
for r in np.arange(0, len(logs), 1):
    
    posterior = posteriors[r]
    samples = rej_sampler(posterior, 5000) # fast parallel rejection sampler
    
    fig = plot_hist_marginals(
                   samples,
                   gt=pars_true, 
                   lims=[-5,2])
    
    fig.set_figheight(12)
    fig.set_figwidth(12)
    fig.suptitle('APT posterior estimates, round r = '+str(r+1), fontsize=14)
    fig.show()
    #print('negative log-probability of ground-truth pars \n', -posterior.eval(pars_true, log=True))